The DCRNN Model is referred by [7]

In [1]:
!git clone https://github.com/liyaguang/DCRNN.git

Cloning into 'DCRNN'...
remote: Enumerating objects: 325, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 325 (delta 27), reused 40 (delta 16), pack-reused 272
Receiving objects: 100% (325/325), 127.90 MiB | 26.79 MiB/s, done.
Resolving deltas: 100% (142/142), done.


Data Preparation

In [2]:
%cd /content/DCRNN/

/content/DCRNN


In [3]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.7.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.7.0


In [4]:
!pip install -r requirements.txt

     |████████████████████████████████| 489.6 MB 23 kB/s 


Find, download, and upload the METR-LA and the PEMS-BAY datasets. Download them to your PC first, then upload them onto the session machine to the left. (/content/DCRNN/data/) In my case, I preuploaded the dataset onto Dropbox

In [12]:
!wget https://www.dropbox.com/s/3uto6j6zba4kcje/metr-la.h5
!wget https://www.dropbox.com/s/akpm0wf7apg6gy0/pems-bay.h5

--2021-12-27 09:41:14--  https://www.dropbox.com/s/3uto6j6zba4kcje/metr-la.h5
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/3uto6j6zba4kcje/metr-la.h5 [following]
--2021-12-27 09:41:14--  https://www.dropbox.com/s/raw/3uto6j6zba4kcje/metr-la.h5
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucebdcf0a272d78b692c918c46a0.dl.dropboxusercontent.com/cd/0/inline/BcqoYFaDoqEQhIyVDeh6Y9pgjly1t_kLMyRqtWGvDxmTlgTI7_lCT5EEzxFuznlLjUJSFEIdUzI7GpwO9HgGCPPqC19-tOUALl_-aVdhqHm3YaUoYOf6UH94cQtAS22-fkxIZVtCpRoDTSXFmKCH7GRA/file# [following]
--2021-12-27 09:41:14--  https://ucebdcf0a272d78b692c918c46a0.dl.dropboxusercontent.com/cd/0/inline/BcqoYFaDoqEQhIyVDeh6Y9pgjly1t_kLMyRqtWGvDxmTlgTI7_lCT5EEzxFuznlLjUJSFEIdUzI7GpwO9HgGCPPqC19-tOUA

Before processing the data, make sure that the downloaded files are in the "data" folder by opening the file explorer tab to the left. If not, move them in.

In [13]:
#%mkdir -p data/{METR-LA,PEMS-BAY}

# METR-LA
!python -m scripts.generate_training_data --output_dir=data/METR-LA --traffic_df_filename=data/metr-la.h5

# PEMS-BAY
!python -m scripts.generate_training_data --output_dir=data/PEMS-BAY --traffic_df_filename=data/pems-bay.h5

Generating training data
tcmalloc: large alloc 1361199104 bytes == 0x5586c4e08000 @  0x7eff5cb931e7 0x7eff5a6d346e 0x7eff5a723c7b 0x7eff5a723d18 0x7eff5a7cb010 0x7eff5a7cb73c 0x7eff5a7cb85d 0x55860c8f3749 0x7eff5a710ef7 0x55860c8f1437 0x55860c8f1240 0x55860c964973 0x55860c95f9ee 0x55860c8f2bda 0x55860c961737 0x55860c95fced 0x55860c831e2b 0x7eff5a710ef7 0x55860c8f1437 0x55860c8f1240 0x55860c964973 0x55860c95f9ee 0x55860c8f2bda 0x55860c961737 0x55860c95f9ee 0x55860c8f2bda 0x55860c961737 0x55860c8f2afa 0x55860c960915 0x55860c8f2afa 0x55860c960915
tcmalloc: large alloc 1361199104 bytes == 0x5587168e8000 @  0x7eff5cb931e7 0x7eff5a6d346e 0x7eff5a723c7b 0x7eff5a723d18 0x7eff5a7cb010 0x7eff5a7cb73c 0x7eff5a7cb85d 0x55860c8f3749 0x7eff5a710ef7 0x55860c8f1437 0x55860c8f1240 0x55860c964973 0x55860c95f9ee 0x55860c8f2bda 0x55860c961737 0x55860c95fced 0x55860c831e2b 0x7eff5a710ef7 0x55860c8f1437 0x55860c8f1240 0x55860c964973 0x55860c95f9ee 0x55860c8f2bda 0x55860c961737 0x55860c95f9ee 0x55860c8f2bda 

After the datasets are loaded, construct a graph as an input for the GCN

In [14]:
!python -m scripts.gen_adj_mx  --sensor_ids_filename=data/sensor_graph/graph_sensor_ids.txt --normalized_k=0.1\
    --output_pkl_filename=data/sensor_graph/adj_mx.pkl

Run the pretrained model

In [15]:
# METR-LA
!python run_demo.py --config_filename=data/model/pretrained/METR-LA/config.yaml

# PEMS-BAY
!python run_demo.py --config_filename=data/model/pretrained/PEMS-BAY/config.yaml
#The generated prediction of DCRNN is in data/results/dcrnn_predictions.

Traceback (most recent call last):
  File "run_demo.py", line 9, in <module>
    from model.dcrnn_supervisor import DCRNNSupervisor
  File "/content/DCRNN/model/dcrnn_supervisor.py", line 16, in <module>
    from model.dcrnn_model import DCRNNModel
  File "/content/DCRNN/model/dcrnn_model.py", line 7, in <module>
    from tensorflow.contrib import legacy_seq2seq
ModuleNotFoundError: No module named 'tensorflow.contrib'
Traceback (most recent call last):
  File "run_demo.py", line 9, in <module>
    from model.dcrnn_supervisor import DCRNNSupervisor
  File "/content/DCRNN/model/dcrnn_supervisor.py", line 16, in <module>
    from model.dcrnn_model import DCRNNModel
  File "/content/DCRNN/model/dcrnn_model.py", line 7, in <module>
    from tensorflow.contrib import legacy_seq2seq
ModuleNotFoundError: No module named 'tensorflow.contrib'


Or retrain the model completely

In [16]:
# METR-LA
!python dcrnn_train.py --config_filename=data/model/dcrnn_la.yaml

# PEMS-BAY
!python dcrnn_train.py --config_filename=data/model/dcrnn_bay.yaml

Traceback (most recent call last):
  File "dcrnn_train.py", line 10, in <module>
    from model.dcrnn_supervisor import DCRNNSupervisor
  File "/content/DCRNN/model/dcrnn_supervisor.py", line 16, in <module>
    from model.dcrnn_model import DCRNNModel
  File "/content/DCRNN/model/dcrnn_model.py", line 7, in <module>
    from tensorflow.contrib import legacy_seq2seq
ModuleNotFoundError: No module named 'tensorflow.contrib'
Traceback (most recent call last):
  File "dcrnn_train.py", line 10, in <module>
    from model.dcrnn_supervisor import DCRNNSupervisor
  File "/content/DCRNN/model/dcrnn_supervisor.py", line 16, in <module>
    from model.dcrnn_model import DCRNNModel
  File "/content/DCRNN/model/dcrnn_model.py", line 7, in <module>
    from tensorflow.contrib import legacy_seq2seq
ModuleNotFoundError: No module named 'tensorflow.contrib'


Eval Baseline Methods

In [17]:
# METR-LA
!python -m scripts.eval_baseline_methods --traffic_reading_filename=data/metr-la.h5

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
2021-12-27 09:49:40,163 - INFO - Log directory: data/model
2021-12-27 09:49:40,438 - INFO - Static
2021-12-27 09:49:40,439 - INFO - Model	Horizon	RMSE	MAPE	MAE
Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/content/DCRNN/scripts/eval_baseline_methods.py", line 141, in <module>
    main(args)
  File "/content/DCRNN/scripts/eval_baseline_methods.py", line 130, in main
    eval_static(traffic_reading_df)
  File "/content/DCRNN/scripts/eval_baseline_methods.py", line 95, in eval_static
    rmse = masked_rmse_np(preds=y_predict.as_matrix(), labels=y_test.as_matrix(), null_val=0)
  File "/usr/loc